# Universidad Peruana de Ciencias Aplicadas
## Topicos de Ciencias de la Computacion - PC4
#### Estudiantes:
####  - Ibrahim Imanol Jordi Arquinigo Jacinto - U20191e650
####  - Ian Joaquin Sanchez Alva - U202124676
####  - Eduardo Jose Rivas Siesquen - U202216407
####  - Daniel Orlando Luis Lazaro - U202021900
####  Nov, 2025

In [1]:
import os
import random
import math
import asyncio
import nest_asyncio
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour

# Aplicar parche para que SPADE funcione en Jupyter
nest_asyncio.apply()

# =============================================================================
# 1. GUI
# =============================================================================
os.makedirs("static", exist_ok=True)

html_content = """
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Minecraft Hardcore Night</title>
    <style>
        body { font-family: 'Segoe UI', monospace; background: #0d0d0d; color: #eee; text-align: center; }
        canvas { background: #1b261b; border: 4px solid #3e2723; margin-top: 10px; box-shadow: 0 0 30px rgba(0,0,0,0.7); }
        .panel { background: #212121; padding: 10px; display: inline-block; border: 1px solid #424242; border-radius: 8px; margin-bottom: 5px; min-width: 800px; }
        b { color: #ffca28; }
        .stat { margin: 0 10px; font-size: 14px; }
        .legend { font-size: 12px; color: #aaa; margin-top: 5px; }
        input[type=range] { vertical-align: middle; accent-color: #d84315; }
    </style>
</head>
<body>
    <div class="panel">
        <span class="stat">📅 Día: <b id="dia">1</b></span> | 
        <span class="stat">💀 Amenaza: <b id="threat" style="color:#ef5350">0</b></span> |
        <span class="stat">👥 Vivos: <b id="pob">0</b></span> |
        <span class="stat">🧟 Horda: <b id="zom">0</b></span> |
        <span class="stat">♻️ Respawn: <b style="color:#00e676">DIARIO</b></span>
        <br><br>
        <label>⏩ Velocidad:</label> <input type="range" id="slider" min="1" max="50" value="1">
    </div>

    <canvas id="simCanvas" width="1000" height="700"></canvas>
    
    <div class="legend">
        🔴 Comida | 🟤 Madera | ⚪ Piedra | 
        <span style="color:#ff9800">🏠 Casa</span> | <span style="color:#29b6f6">Steve</span>
    </div>

    <script>
        const canvas = document.getElementById('simCanvas');
        const ctx = canvas.getContext('2d');
        const ANCHO = 1000, ALTO = 700;

        function draw(data) {
            ctx.fillStyle = data.is_night ? "#05040a" : "#2e7d32"; // Noche más oscura
            ctx.fillRect(0, 0, ANCHO, ALTO);

            // Recursos
            data.recursos.forEach(r => {
                if(r.type === 'food') { 
                    ctx.fillStyle = '#ff1744'; ctx.beginPath(); ctx.arc(r.x, r.y, 4, 0, Math.PI*2); ctx.fill(); 
                } else {
                    ctx.fillStyle = (r.type === 'wood') ? '#795548' : '#bdbdbd'; 
                    ctx.fillRect(r.x-3, r.y-3, 7, 7);
                }
            });

            // Casas
            data.casas.forEach(h => {
                let size = 16 + (h.lvl * 6);
                ctx.fillStyle = h.lvl === 3 ? "#bf360c" : (h.lvl === 2 ? "#e65100" : "#ff9800");
                ctx.fillRect(h.x - size/2, h.y - size/2, size, size);
                // Barra HP Casa
                ctx.fillStyle = "black"; ctx.fillRect(h.x - size/2, h.y - size/2 - 6, size, 4);
                ctx.fillStyle = "#00e676"; ctx.fillRect(h.x - size/2, h.y - size/2 - 6, size * (h.hp/h.max_hp), 4);
                // Ocupantes
                ctx.fillStyle = "white"; ctx.font = "10px monospace";
                ctx.fillText(h.occupants + "/" + h.cap, h.x - 10, h.y + 4);
            });

            // Steves
            data.steves.forEach(p => {
                ctx.beginPath();
                ctx.fillStyle = "#29b6f6"; 
                ctx.arc(p.x, p.y, 6, 0, Math.PI*2);
                ctx.fill();
                // Barra HP Steve
                ctx.fillStyle = "red"; ctx.fillRect(p.x-6, p.y-9, 12, 2);
                ctx.fillStyle = "#76ff03"; ctx.fillRect(p.x-6, p.y-9, 12 * (p.hp/100), 2);
            });

            // Zombies
            data.zombies.forEach(z => {
                ctx.beginPath();
                ctx.fillStyle = "#1b5e20"; 
                ctx.arc(z.x, z.y, 7 + (z.tier * 2), 0, Math.PI*2);
                ctx.fill();
                ctx.strokeStyle = "red"; ctx.lineWidth = 1; ctx.stroke();
            });

            document.getElementById('dia').innerText = data.dia;
            document.getElementById('threat').innerText = data.difficulty;
            document.getElementById('pob').innerText = data.steves.length;
            document.getElementById('zom').innerText = data.zombies.length;
        }

        async function loop() {
            try {
                const response = await fetch('/data'); 
                if (response.ok) draw(await response.json());
            } catch(e) {}
            setTimeout(loop, 50); 
        }

        document.getElementById('slider').oninput = async function() {
            await fetch('/speed', { method: 'POST', body: JSON.stringify({val: this.value}) });
        };
        loop();
    </script>
</body>
</html>
"""
with open("static/index.html", "w", encoding='utf-8') as f: f.write(html_content)
print("✅ GUI Generada.")

# =============================================================================
# 2. DEFINICIÓN DEL MUNDO (CAMBIOS AQUÍ)
# =============================================================================
ANCHO = 1000
ALTO = 700

# --- MODIFICADO: NOCHE LARGA ---
DURACION_DIA = 200      # Día corto
DURACION_NOCHE = 400    # Noche Larga (El doble)
CICLO_TOTAL = DURACION_DIA + DURACION_NOCHE

class Recurso:
    def __init__(self, tipo):
        self.x = random.randint(20, ANCHO - 20)
        self.y = random.randint(20, ALTO - 20)
        self.type = tipo 

class Casa:
    def __init__(self, x, y, owner_id):
        self.x = x; self.y = y
        self.owner_id = owner_id
        self.lvl = 1; self.cap = 1; self.occupants = 0
        self.hp = 200; self.max_hp = 200

# =============================================================================
# 3. IMPLEMENTACIÓN DE LOS AGENTES
# =============================================================================

class SteveAgent(Agent):
    def __init__(self, jid, password, genes=None):
        super().__init__(jid, password)
        self.x = random.randint(50, ANCHO-50)
        self.y = random.randint(50, ALTO-50)
        self.hp = 100
        self.wood = 0; self.stone = 0; self.food = 0 
        self.energy = 60 
        self.inside_house = None 
        
        if genes:
            self.g_flee = genes['flee']
            self.g_work = genes['work']
        else:
            self.g_flee = random.randint(50, 250)
            self.g_work = random.randint(1, 10)

    async def setup(self): pass

    def perceive_and_act(self, mgr):
        if self.hp <= 0: return

        # --- SISTEMA DE HAMBRE ---
        if mgr.tick % 30 == 0: self.energy -= 5 
        if self.energy <= 0: self.hp -= 5 
        
        # Comer
        if self.energy < 50 and self.food > 0:
            self.food -= 1; self.energy += 40; self.hp = min(100, self.hp + 20)

        # 1. LÓGICA DENTRO DE CASA (MEJORAR Y REFUGIARSE)
        if self.inside_house:
            if self.inside_house not in mgr.houses:
                self.inside_house = None
                return
            h = self.inside_house
            self.hp = min(100, self.hp + 2) 
            
            # >>> FIX: MEJORA DE CASA MÁS BARATA <<<
            if h.lvl < 3:
                # Costo reducido: 4 madera/piedra para Lvl 2, 8 para Lvl 3
                cost = h.lvl * 4 
                if self.wood >= cost and self.stone >= cost:
                    self.wood -= cost; self.stone -= cost
                    h.lvl += 1
                    h.cap = 3 if h.lvl == 2 else 5
                    h.max_hp += 300 # Aumentar vida máxima
                    h.hp = h.max_hp # ¡Reparar casa al mejorarla!
                    self.energy += 30 
            # ----------------------------------------
            
            if mgr.is_night: return
            
            # Salir si tengo hambre o energía para trabajar
            if (self.energy > 40 and self.food == 0) or self.energy > 30: 
                 h.occupants -= 1; self.inside_house = None

        # 2. ZOMBIES (HUIR)
        closest_z = None; min_d = 999
        for z in mgr.zombies:
            d = math.hypot(z.x - self.x, z.y - self.y)
            if d < min_d: min_d = d; closest_z = z
        
        if min_d < self.g_flee:
            dx = self.x - closest_z.x; dy = self.y - closest_z.y
            self.move(dx, dy, 5.0)
            return

        # 3. NOCHE (BUSCAR REFUGIO)
        if mgr.is_night:
            best_h = None; min_d_h = 999
            for h in mgr.houses:
                if h.occupants < h.cap: 
                    d = math.hypot(h.x - self.x, h.y - self.y)
                    if d < min_d_h: min_d_h = d; best_h = h
            if best_h:
                if math.hypot(best_h.x - self.x, best_h.y - self.y) < 10:
                    self.inside_house = best_h; best_h.occupants += 1 
                else:
                    self.move(best_h.x - self.x, best_h.y - self.y, 4.5)
            else:
                self.move(random.uniform(-1,1), random.uniform(-1,1), 3.0)
            return

        # 4. DÍA (TRABAJAR)
        target_type = 'food'
        # Si tengo energía, priorizo materiales para poder mejorar la casa
        if self.energy > 40 and self.food > 0:
             target_type = 'wood' if self.wood <= self.stone else 'stone'
        
        cost = 5
        if self.wood >= cost and self.stone >= cost and target_type != 'food':
            if random.randint(0,10) < self.g_work:
                new_h = Casa(self.x, self.y, str(self.jid))
                mgr.houses.append(new_h)
                self.inside_house = new_h; new_h.occupants = 1 
                self.wood -= cost; self.stone -= cost
                self.energy += 10
                return

        closest_r = None; min_dist = 999
        for r in mgr.resources:
            if r.type == target_type:
                d = math.hypot(r.x - self.x, r.y - self.y)
                if d < min_dist: min_dist = d; closest_r = r
        
        if closest_r:
            if math.hypot(closest_r.x - self.x, closest_r.y - self.y) < 8:
                if closest_r in mgr.resources:
                    mgr.resources.remove(closest_r)
                    if closest_r.type == 'food': 
                        self.food += 1
                        if self.energy < 20: self.food -= 1; self.energy += 40
                    elif closest_r.type == 'wood': self.wood += 1
                    else: self.stone += 1
            else:
                self.move(closest_r.x - self.x, closest_r.y - self.y, 3.5)
        else:
            speed = 4.0 if target_type == 'food' else 1.5
            self.move(random.uniform(-1,1), random.uniform(-1,1), speed)

    def move(self, dx, dy, s):
        d = math.hypot(dx, dy)
        if d > 0: self.x += (dx/d)*s; self.y += (dy/d)*s
        self.x = max(0, min(ANCHO, self.x)); self.y = max(0, min(ALTO, self.y))

class ZombieObject:
    def __init__(self, difficulty):
        side = random.randint(0,3)
        if side==0: self.x, self.y = random.randint(0,ANCHO), 0
        elif side==1: self.x, self.y = random.randint(0,ANCHO), ALTO
        elif side==2: self.x, self.y = 0, random.randint(0,ALTO)
        else: self.x, self.y = ANCHO, random.randint(0,ALTO)
        
        self.tier = 0 
        self.speed = 2.0 + (difficulty * 0.2)
        self.damage = 1.0 + (difficulty * 0.5)

    def logic(self, mgr):
        target = None; min_d = 999
        
        for s in mgr.steves:
            if s.inside_house: continue 
            d = math.hypot(s.x - self.x, s.y - self.y)
            if d < min_d: min_d = d; target = s
        
        for h in mgr.houses:
            d = math.hypot(h.x - self.x, h.y - self.y)
            prio = d - (500 if h.occupants > 0 else 0) 
            if prio < min_d: min_d = prio; target = h

        if target:
            dx = target.x - self.x; dy = target.y - self.y
            dist = math.hypot(dx, dy)
            if dist > 0:
                self.x += (dx/dist) * self.speed; self.y += (dy/dist) * self.speed
            
            if dist < 12:
                if isinstance(target, SteveAgent):
                    target.hp -= self.damage
                    if target.hp <= 0:
                        self.tier += 1; self.speed += 0.5; self.damage += 2; target.hp = 0
                elif isinstance(target, Casa):
                    # >>> FIX: NERF AL DAÑO ESTRUCTURAL <<<
                    # Antes era * 2. Ahora es * 0.5 (Hacen menos daño a las paredes)
                    target.hp -= self.damage * 0.5 
                    if target.hp <= 0:
                        if target in mgr.houses: mgr.houses.remove(target)
                        for s in mgr.steves: 
                            if s.inside_house == target: s.inside_house = None

# =============================================================================
# 4. MANAGER
# =============================================================================

class ManagerAgent(Agent):
    async def setup(self):
        print("🎮 Servidor Minecraft Hardcore Iniciado.")
        self.steves = [SteveAgent(f"Gen0_{i}@loc", "pass") for i in range(25)]
        self.zombies = []
        self.houses = []
        self.resources = []
        self.respawn_resources()
        
        self.gen = 0; self.day = 1; self.tick = 0
        self.difficulty = 0
        self.is_night = False
        self.delay = 0.05

        self.web.start(port=10000)
        self.web.app.router.add_static("/static", "static")
        self.web.add_get("/data", self.send_data, template=None)
        self.web.add_post("/speed", self.set_speed, template=None)
        self.add_behaviour(self.WorldLoop())

    def respawn_resources(self):
        # Llenamos TODO, pero un poco menos de comida para que sufran
        self.resources = []
        for _ in range(120): self.resources.append(Recurso('food')) 
        for _ in range(100): self.resources.append(Recurso('wood'))
        for _ in range(100): self.resources.append(Recurso('stone'))
        print("✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!")

    async def send_data(self, req):
        return {
            "gen": self.gen, "dia": self.day, "is_night": self.is_night, 
            "difficulty": self.difficulty, 
            "steves": [{"x":s.x,"y":s.y,"hp":s.hp,"has_sword":(s.wood>5), "flee_dist":s.g_flee} for s in self.steves],
            "zombies": [{"x":z.x,"y":z.y,"tier":z.tier} for z in self.zombies],
            "recursos": [{"x":r.x,"y":r.y,"type":r.type} for r in self.resources],
            "casas": [{"x":h.x,"y":h.y,"hp":h.hp,"max_hp":h.max_hp,"lvl":h.lvl,"cap":h.cap,"occupants":h.occupants} for h in self.houses]
        }
    
    async def set_speed(self, req):
        self.delay = 0.05 / float((await req.json()).get('val',1))
        return {"status":"ok"}

    class WorldLoop(CyclicBehaviour):
        async def run(self):
            mgr = self.agent
            cycle = mgr.tick % CICLO_TOTAL
            mgr.is_night = cycle > DURACION_DIA
            
            # --- NUEVO DÍA ---
            if cycle == 0:
                mgr.day += 1
                mgr.zombies = [] 
                
                # --- MODIFICADO: RESPAWN DIARIO ---
                # Ya no esperamos 5 días. Sale comida nueva CADA DÍA.
                mgr.respawn_resources()
                
                if mgr.day % 2 == 0:
                    mgr.difficulty += 1 
                
                # Reproducción
                survivors = [s for s in mgr.steves if s.hp > 0]
                if survivors:
                    parents = [s for s in survivors if s.energy > 50] 
                    if not parents: parents = survivors[:2] 
                    
                    next_gen = []
                    for p in survivors: 
                        p.hp=100; p.energy=60; p.inside_house=None; p.x=random.randint(50,ANCHO-50)
                        next_gen.append(p)
                    
                    for p in parents:
                        if len(next_gen) < 15: 
                            genes = {'flee': p.g_flee+random.randint(-15,15), 'work': p.g_work+random.randint(-1,1)}
                            next_gen.append(SteveAgent(f"G{mgr.gen}_{len(next_gen)}@l","p", genes))
                    
                    mgr.steves = next_gen
                    mgr.gen += 1
                else:
                    print("💀 GAME OVER.")
                    mgr.steves = [SteveAgent(f"Adam_{i}@l","p") for i in range(25)]
                    mgr.respawn_resources()
                    mgr.gen = 0; mgr.difficulty = 0

            # --- ZOMBIE SPAWN MASIVO ---
            max_zom = 12 + (mgr.difficulty * 5)
            if mgr.is_night and len(mgr.zombies) < max_zom and random.random()<0.15:
                mgr.zombies.append(ZombieObject(mgr.difficulty))

            # --- UPDATE ---
            active_steves = []
            for s in mgr.steves:
                s.perceive_and_act(mgr)
                if s.hp > 0: active_steves.append(s)
            mgr.steves = active_steves
            
            for z in mgr.zombies: z.logic(mgr)

            mgr.tick += 1
            await asyncio.sleep(getattr(mgr, 'delay', 0.05))

async def main():
    manager = ManagerAgent("admin@localhost", "password")
    await manager.start()
    print("🚀 Minecraft EVO Hardcore (Noche Larga) Corriendo...")
    try:
        while True: await asyncio.sleep(1)
    except KeyboardInterrupt: await manager.stop()

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    if loop.is_running(): loop.create_task(main())
    else: asyncio.run(main())

✅ GUI Generada.


🎮 Servidor Minecraft Hardcore Iniciado.
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
🚀 Minecraft EVO Hardcore (Noche Larga) Corriendo...
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (RESPAWN DIARIO)!
✨ ¡RECURSOS RESTAURADOS (R